In [22]:
import pandas as pd 
import os, fnmatch
import sys
import numpy as np

# bts_df = pd.DataFrame()
# listOfFolders= os.listdir('./raw_data')
# print(listOfFolders)
# pattern = "*.csv"
# Check every folder in the list of folders



In [2]:
# for folder in listOfFolders:
#     if not folder.startswith("."): 
#         listOfFiles = os.listdir('./raw_data/{}'.format(folder))

#         # Check everyfile in that folder
#         for files in listOfFiles:
#             try:
#                 if fnmatch.fnmatch(files, pattern):
#                     current_fn = "./raw_data/{}/{}".format(folder, files)
#                     cur_df = pd.read_csv(current_fn)
#                     bts_df = bts_df.append(cur_df)
#                     print("processing: {}".format(current_fn))
#                     bts_df = bts_df.dropna(subset=['reading_time'])
#                     bts_df["unix_timestamp"] = pd.to_datetime(bts_df["reading_time"]).astype(int)
#                     mean_time = bts_df["unix_timestamp"].mean()
#                     min_time = bts_df["unix_timestamp"].min()
#                     bts_df["norm_time"] = (bts_df["unix_timestamp"]-mean_time)/(3600*1000000000)
#                     bts_df = bts_df.sort_values(by=['norm_time'])
#                     bts_df.drop(["reading_time"], axis='columns', inplace=True)

#                     bts_df_grouped = bts_df.groupby(["station_id","parameter_id"])
#                     for key,item in bts_df_grouped:
#                         if key[1] == 122 and key[0] == 1161114004:
#                             sub_data = bts_df_grouped.get_group(key)
# #                             mean_val = sub_data['value'].mean()
# #                             sub_data['norm_value'] = sub_data['value']-mean_val
# #                             max_val = sub_data['norm_value'].max()
# #                             sub_data['norm_value'] = sub_data['norm_value']/max_val
#                             sub_data.sort_values(by=['norm_time']).to_csv("./grouped_data/{}_{}_{}_.csv".format(folder, key[0],key[1]), index=False)
#                             print("Finish: {}".format(key))
#             except Exception as e:
#                 print(e)

In [18]:
daily_prediction_result = pd.read_csv("Result/12_05_21.csv", names=["id","value","station_id","parameter_id","unix_timestamp","label", "prediction"])



In [19]:
daily_prediction_result

,id,value,station_id,parameter_id,unix_timestamp,label,prediction
0,5.706646e+09,8.0,1.161114e+09,122.0,1.493514e+18,-0.477082,-0.956165
1,5.706648e+09,9.0,1.161114e+09,122.0,1.493514e+18,-0.467082,-0.956165
2,5.706648e+09,10.0,1.161114e+09,122.0,1.493514e+18,-0.466804,-0.956165
3,5.706648e+09,11.5,1.161114e+09,122.0,1.493514e+18,-0.466526,-0.956165
4,5.706648e+09,14.0,1.161114e+09,122.0,1.493514e+18,-0.466248,-0.956165
...,...,...,...,...,...,...,...
187,5.706813e+09,15.0,1.161114e+09,122.0,1.493517e+18,0.434863,0.298047
188,5.706814e+09,16.0,1.161114e+09,122.0,1.493517e+18,0.441807,0.394910
189,5.706815e+09,16.0,1.161114e+09,122.0,1.493517e+18,0.448196,0.394910
190,5.706817e+09,15.0,1.161114e+09,122.0,1.493517e+18,0.457363,0.298047


In [24]:
y_true = np.array(daily_prediction_result["label"])
y_pred = np.array(daily_prediction_result["prediction"])

In [25]:

# Mean Squared Error
MSE = np.square(np.subtract(y_true,y_pred)).mean()
MSE

0.17581912213640574

In [34]:
    # Read dataset from file
    raw_dataset = pd.read_csv("/Users/nguyenlinh/Macadamia/Advanced_SS_project/MLService/Result/12_05_21.csv", 
        names=["id","value","station_id","parameter_id","unix_timestamp","label", "prediction"])
    mean_val = raw_dataset['value'].mean()
    raw_dataset['norm_value'] = raw_dataset['value']-mean_val
    max_val = raw_dataset['norm_value'].max()
    raw_dataset['norm_value'] = raw_dataset['norm_value']/max_val
    raw_dataset = raw_dataset.astype({'id':'float','value':'float', 'station_id':'int', 'parameter_id':'int', 'unix_timestamp':'int', 'label':'float', 'prediction':'float'})
    
    dataset = raw_dataset.copy()
    dataset = dataset.dropna().drop(['id','station_id','parameter_id','unix_timestamp'], axis=1)
    dataset_full = dataset.sort_values(by=['norm_time'])
    dataset = dataset_full[0:300]

    # pre-processing data 

    serial_data = dataset.drop(['value','unix_timestamp'], axis=1)
    serial_data['norm_1'] = serial_data['norm_value'].shift(1)
    serial_data['norm_2'] = serial_data['norm_value'].shift(2)
    serial_data['norm_3'] = serial_data['norm_value'].shift(3)
    serial_data['norm_4'] = serial_data['norm_value'].shift(4)
    serial_data['norm_5'] = serial_data['norm_value'].shift(5)
    serial_data['norm_6'] = serial_data['norm_value'].shift(6)
    serial_data = serial_data[6:]

   # Split data into training and testing
    train_dataset = serial_data.sample(frac=0.8, random_state=1)
    test_dataset = serial_data.drop(train_dataset.index)

    train_features = np.array(train_dataset.drop(['norm_value'], axis=1))
    train_features = np.array(train_features)[:,:,np.newaxis]
    train_labels = np.array(train_dataset.drop(['norm_6'], axis=1))
    train_labels = train_labels.reshape(train_labels.shape[0],train_labels.shape[1],1)
    
    test_features = np.array(test_dataset.drop(['norm_value'], axis=1))
    test_features = test_features.reshape(test_features.shape[0],test_features.shape[1],1)
    test_features.shape

    model = keras.Sequential()
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.TimeDistributed(layers.Dense(1)))
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.005))
    model.fit(train_features, train_labels, epochs=1, batch_size=1, verbose=2)
    # load saved_model 
    interpreter = tflite.Interpreter("./exported_models/tflite_model/LSTM_single_series.tflite")
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    # interpreter.resize_tensor_input(input_details[0]['index'],batch_input)
    # interpreter.resize_tensor_input(output_details[0]['index'],batch_input)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.allocate_tensors()
    interpreter.set_tensor(input_details[0]['index'], test_features)
    interpreter.invoke()
    old_model_result = interpreter.get_tensor(output_details[0]['index']) 


KeyError: 'norm_time'

In [30]:
raw_dataset

,id,value,station_id,parameter_id,unix_timestamp,label,prediction,norm_value
0,5.706646e+09,8.0,1161114004,122,1493514070000000000,-0.477082,-0.956165,-0.906653
1,5.706648e+09,9.0,1161114004,122,1493514106000000000,-0.467082,-0.956165,-0.715988
2,5.706648e+09,10.0,1161114004,122,1493514107000000000,-0.466804,-0.956165,-0.525323
3,5.706648e+09,11.5,1161114004,122,1493514108000000000,-0.466526,-0.956165,-0.239325
4,5.706648e+09,14.0,1161114004,122,1493514109000000000,-0.466248,-0.956165,0.237339
...,...,...,...,...,...,...,...,...
187,5.706813e+09,15.0,1161114004,122,1493517353000000000,0.434863,0.298047,0.428004
188,5.706814e+09,16.0,1161114004,122,1493517378000000000,0.441807,0.394910,0.618669
189,5.706815e+09,16.0,1161114004,122,1493517401000000000,0.448196,0.394910,0.618669
190,5.706817e+09,15.0,1161114004,122,1493517434000000000,0.457363,0.298047,0.428004
